In [ ]:
## law_1.docx, law_2.docx
- pinecone store 저장
    - index name: 임의
- RetrievalQA 구현
    - prompt: rlm/rag-prompt
    - 질문: 전세사기

In [ ]:
import os

from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import Docx2txtLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone


load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'laws-index'
pc = Pinecone(api_key=PINECONE_API_KEY)

## 문서 파일 목록
doc_paths = ['law_1.docx', 'law_2.docx']

## 문서 로드(읽어오기)
documents = []

for path in doc_paths :
    loader = Docx2txtLoader(path)
    documents.extend(loader.load())

## 청크 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

## 문서 분할
document_list = text_splitter.split_documents(documents)

database = PineconeVectorStore.from_documents(
    index_name=index_name,
    embedding=embedding,
    documents=document_list,    
)

llm = ChatOpenAI()
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt 
    | llm    
    | StrOutputParser() 
)

qa_chain.invoke('전세사기 피해주택 임대인의 국세는 어떻게 계산하나요? 정확한 수식을 알려주세요')

In [2]:
# 저장없이 데이터베이스를 읽기만 할때

import os

from dotenv import load_dotenv
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone


## 환경변수 읽어오기 ############################################
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')

## 벡터 스토어(데이터베이스)에서 인덱스 가져오기 ################
## 임베딩 모델 지정
embedding = OpenAIEmbeddings(model='text-embedding-3-large')
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = 'laws-index'

## 저장된 인덱스 가져오기 #######################################
database = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding,
)

## RetrievalQA ##################################################
llm = ChatOpenAI(model='gpt-4o')
prompt = hub.pull('rlm/rag-prompt')

def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

qa_chain = (
    {
        'context': database.as_retriever() | format_docs,
        'question': RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

qa_chain.invoke('전세사기 피해주택 임대인의 국세는 어떻게 계산하나요? 정확한 수식을 알려주세요')

'전세사기 피해주택 임대인의 국세는 상속세, 증여세, 및 종합부동산세의 경우 다음의 수식으로 계산됩니다: A × (B / C). 여기서 A는 임대인이 체납한 국세 금액, B는 피해주택의 가격, C는 임대인이 보유한 모든 주택 가격의 합계액입니다. 이 방식은 각각의 고지 또는 신고 건별로 산정됩니다.'